In [9]:
#导入前置依赖
import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
# 用于加载bert模型的分词器
from transformers import AutoTokenizer
# 用于加载bert模型
from transformers import BertModel
from pathlib import Path



In [10]:
batch_size = 16
# 文本的最大长度
text_max_length = 128
# 总训练的epochs数，我只是随便定义了个数
epochs = 3
# 学习率
lr = 3e-5
# 取多少训练集的数据作为验证集
validation_ratio = 0.1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 每多少步，打印一次loss
log_per_step = 50

# 数据集所在位置
dataset_dir = Path("./data")
os.makedirs(dataset_dir) if not os.path.exists(dataset_dir) else ''

# 模型存储路径
model_dir = Path("./model/bert_checkpoints")
# 如果模型目录不存在，则创建一个
os.makedirs(model_dir) if not os.path.exists(model_dir) else ''

print("Device:", device)



Device: cpu


In [11]:
# 读取数据集，进行数据处理

pd_train_data = pd.read_csv('./data/train.csv')
pd_train_data['title'] = pd_train_data['title'].fillna('')
pd_train_data['abstract'] = pd_train_data['abstract'].fillna('')

test_data = pd.read_csv('./data/testB.csv')
test_data['title'] = test_data['title'].fillna('')
test_data['abstract'] = test_data['abstract'].fillna('')
pd_train_data['text'] = pd_train_data['title'].fillna('') + ' ' +  pd_train_data['author'].fillna('') + ' ' + pd_train_data['abstract'].fillna('')+ ' ' + pd_train_data['Keywords'].fillna('')
test_data['text'] = test_data['title'].fillna('') + ' ' +  test_data['author'].fillna('') + ' ' + test_data['abstract'].fillna('')+ ' ' + pd_train_data['Keywords'].fillna('')


In [12]:
# 从训练集中随机采样测试集
validation_data = pd_train_data.sample(frac=validation_ratio)
train_data = pd_train_data[~pd_train_data.index.isin(validation_data.index)]

In [13]:
# 构建Dataset
class MyDataset(Dataset):

    def __init__(self, mode='train'):
        super(MyDataset, self).__init__()
        self.mode = mode
        # 拿到对应的数据
        if mode == 'train':
            self.dataset = train_data
        elif mode == 'validation':
            self.dataset = validation_data
        elif mode == 'test':
            # 如果是测试模式，则返回内容和uuid。拿uuid做target主要是方便后面写入结果。
            self.dataset = test_data
        else:
            raise Exception("Unknown mode {}".format(mode))

    def __getitem__(self, index):
        # 取第index条
        data = self.dataset.iloc[index]
        # 取其内容
        text = data['text']
        # 根据状态返回内容
        if self.mode == 'test':
            # 如果是test，将uuid做为target
            label = data['uuid']
        else:
            label = data['label']
        # 返回内容和label
        return text, label

    def __len__(self):
        return len(self.dataset)


In [14]:
train_dataset = MyDataset('train')
validation_dataset = MyDataset('validation')

In [15]:
train_dataset.__getitem__(0)


('Accessible Visual Artworks for Blind and Visually Impaired People: Comparing a Multimodal Approach with Tactile Graphics Quero, Luis Cavazos; Bartolome, Jorge Iranzo; Cho, Jundong Despite the use of tactile graphics and audio guides, blind and visually impaired people still face challenges to experience and understand visual artworks independently at art exhibitions. Art museums and other art places are increasingly exploring the use of interactive guides to make their collections more accessible. In this work, we describe our approach to an interactive multimodal guide prototype that uses audio and tactile modalities to improve the autonomous access to information and experience of visual artworks. The prototype is composed of a touch-sensitive 2.5D artwork relief model that can be freely explored by touch. Users can access localized verbal descriptions and audio by performing touch gestures on the surface while listening to themed background music along. We present the design requi

In [8]:
#获取Bert预训练模型
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [9]:
#接着构造我们的Dataloader。
#我们需要定义一下collate_fn，在其中完成对句子进行编码、填充、组装batch等动作：
def collate_fn(batch):
    """
    将一个batch的文本句子转成tensor，并组成batch。
    :param batch: 一个batch的句子，例如: [('推文', target), ('推文', target), ...]
    :return: 处理后的结果，例如：
             src: {'input_ids': tensor([[ 101, ..., 102, 0, 0, ...], ...]), 'attention_mask': tensor([[1, ..., 1, 0, ...], ...])}
             target：[1, 1, 0, ...]
    """
    text, label = zip(*batch)
    text, label = list(text), list(label)

    # src是要送给bert的，所以不需要特殊处理，直接用tokenizer的结果即可
    # padding='max_length' 不够长度的进行填充
    # truncation=True 长度过长的进行裁剪
    src = tokenizer(text, padding='max_length', max_length=text_max_length, return_tensors='pt', truncation=True)

    return src, torch.LongTensor(label)

In [10]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [11]:
inputs, targets = next(iter(train_loader))
print("inputs:", inputs)
print("targets:", targets)

inputs: {'input_ids': tensor([[  101,  2047,  4245,  ...,     0,     0,     0],
        [  101,  2110,  1011,  ...,  2988,  2913,   102],
        [  101,  8161, 12702,  ...,  2572,  8189,   102],
        ...,
        [  101,  2659,  1011,  ..., 17779, 12823,   102],
        [  101, 20077,  2309,  ...,  9610,  5283,   102],
        [  101, 24155,  1997,  ..., 20875,  5072,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}
targets: tensor([0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1])


In [12]:
#定义预测模型，该模型由bert模型加上最后的预测层组成
class MyModel(nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()

        # 加载bert模型
        self.bert = BertModel.from_pretrained('bert-base-uncased', mirror='tuna')

        # 最后的预测层
        self.predictor = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, src):
        """
        :param src: 分词后的推文数据
        """

        # 将src直接序列解包传入bert，因为bert和tokenizer是一套的，所以可以这么做。
        # 得到encoder的输出，用最前面[CLS]的输出作为最终线性层的输入
        outputs = self.bert(**src).last_hidden_state[:, 0, :]

        # 使用线性层来做最终的预测
        return self.predictor(outputs)


In [13]:
model = MyModel()
model = model.to(device)

In [14]:
#定义出损失函数和优化器。这里使用Binary Cross Entropy：
criteria = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


In [15]:
# 由于inputs是字典类型的，定义一个辅助函数帮助to(device)
def to_device(dict_tensors):
    result_tensors = {}
    for key, value in dict_tensors.items():
        result_tensors[key] = value.to(device)
    return result_tensors

In [16]:
#定义一个验证方法，获取到验证集的精准率和loss
def validate():
    model.eval()
    total_loss = 0.
    total_correct = 0
    for inputs, targets in validation_loader:
        inputs, targets = to_device(inputs), targets.to(device)
        outputs = model(inputs)
        loss = criteria(outputs.view(-1), targets.float())
        total_loss += float(loss)

        correct_num = (((outputs >= 0.5).float() * 1).flatten() == targets).sum()
        total_correct += correct_num

    return total_correct / len(validation_dataset), total_loss / len(validation_dataset)


In [17]:
# 13mins 1个epoch


# 首先将模型调成训练模式
model.train()

# 清空一下cuda缓存
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# 定义几个变量，帮助打印loss
total_loss = 0.
# 记录步数
step = 0

# 记录在验证集上最好的准确率
best_accuracy = 0

# 开始训练
for epoch in range(epochs):
    model.train()
    for i, (inputs, targets) in enumerate(train_loader):
        # 从batch中拿到训练数据
        inputs, targets = to_device(inputs), targets.to(device)
        # 传入模型进行前向传递
        outputs = model(inputs)
        # 计算损失
        loss = criteria(outputs.view(-1), targets.float())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += float(loss)
        step += 1

        if step % log_per_step == 0:
            print("Epoch {}/{}, Step: {}/{}, total loss:{:.4f}".format(epoch+1, epochs, i, len(train_loader), total_loss))
            total_loss = 0

        del inputs, targets

    # 一个epoch后，使用过验证集进行验证
    accuracy, validation_loss = validate()
    print("Epoch {}, accuracy: {:.4f}, validation loss: {:.4f}".format(epoch+1, accuracy, validation_loss))
    torch.save(model, model_dir / f"model_{epoch}.pt")

    # 保存最好的模型
    if accuracy > best_accuracy:
        torch.save(model, model_dir / f"model_best.pt")
        best_accuracy = accuracy

Epoch 1/3, Step: 49/338, total loss:15.1095
Epoch 1/3, Step: 99/338, total loss:7.2713
Epoch 1/3, Step: 149/338, total loss:6.0062
Epoch 1/3, Step: 199/338, total loss:6.4491
Epoch 1/3, Step: 249/338, total loss:4.3200
Epoch 1/3, Step: 299/338, total loss:3.6696
Epoch 1, accuracy: 0.9650, validation loss: 0.0074
Epoch 2/3, Step: 11/338, total loss:3.5065
Epoch 2/3, Step: 61/338, total loss:3.6303
Epoch 2/3, Step: 111/338, total loss:2.7172
Epoch 2/3, Step: 161/338, total loss:4.0392
Epoch 2/3, Step: 211/338, total loss:4.0982
Epoch 2/3, Step: 261/338, total loss:4.3381
Epoch 2/3, Step: 311/338, total loss:3.9734
Epoch 2, accuracy: 0.9650, validation loss: 0.0063
Epoch 3/3, Step: 23/338, total loss:2.7255
Epoch 3/3, Step: 73/338, total loss:3.0730
Epoch 3/3, Step: 123/338, total loss:2.7901
Epoch 3/3, Step: 173/338, total loss:2.5105
Epoch 3/3, Step: 223/338, total loss:2.0831
Epoch 3/3, Step: 273/338, total loss:1.2892
Epoch 3/3, Step: 323/338, total loss:3.6629
Epoch 3, accuracy: 0.96

In [18]:
#加载最好的模型，然后进行测试集的预测
model = torch.load(model_dir / f"model_best.pt")
model = model.eval()


In [19]:
test_dataset = MyDataset('test')
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [20]:
results = []
for inputs, ids in test_loader:
    outputs = model(inputs.to(device))
    outputs = (outputs >= 0.5).int().flatten().tolist()
    ids = ids.tolist()
    results = results + [(id, result) for result, id in zip(outputs, ids)]
    

In [17]:
test_label = [pair[1] for pair in results]
test_data['label'] = test_label
test_data[['uuid', 'label']].to_csv('submit_task1.csv', index=None)



NameError: name 'results' is not defined

In [22]:

# endthis


## Task2




In [4]:
# 导入pandas用于读取表格数据
import pandas as pd

# 导入BOW（词袋模型），可以选择将CountVectorizer替换为TfidfVectorizer（TF-IDF（词频-逆文档频率）），注意上下文要同时修改，亲测后者效果更佳
from sklearn.feature_extraction.text import TfidfVectorizer
# 导入Bert模型
from sentence_transformers import SentenceTransformer

# 导入计算相似度前置库，为了计算候选者和文档之间的相似度，我们将使用向量之间的余弦相似度，因为它在高维度下表现得相当好。
from sklearn.metrics.pairwise import cosine_similarity

# 过滤警告消息
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)


In [18]:
# 读取数据集
test = pd.read_csv('./data/testB.csv')
test['title'] = test['title'].fillna('')
test['abstract'] = test['abstract'].fillna('')

test['text'] = test['title'].fillna('') + ' ' +test['abstract'].fillna('')


In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gwj1139/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/gwj1139/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:

# tokenizers
# stops

# 定义停用词，去掉出现较多，但对文章不关键的词语
stops =[i.strip() for i in open(r'stop.txt',encoding='utf-8').readlines()] 


In [5]:
model = SentenceTransformer(r'xlm-r-distilroberta-base-paraphrase-v1')

In [19]:
# 18mins

test_words = []
for row in test.iterrows():
    # 读取第每一行数据的标题与摘要并提取关键词
    # 修改n_gram_range来改变结果候选词的词长大小。例如，如果我们将它设置为(3，3)，那么产生的候选词将是包含3个关键词的短语。
    n_gram_range = (1,3)
    # 这里我们使用TF-IDF算法来获取候选关键词 
    count = TfidfVectorizer(ngram_range=n_gram_range, stop_words=stops).fit([row[1].text])
    candidates = count.get_feature_names_out()
    # 将文本标题以及候选关键词/关键短语转换为数值型数据（numerical data）。我们使用BERT来实现这一目的
    title_embedding = model.encode([row[1].title])
    
    candidate_embeddings = model.encode(candidates)
    
    # 通过修改这个参数来更改关键词数量
    top_n = 40
    # 利用文章标题进一步提取关键词
    distances = cosine_similarity(title_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
    
    if len( keywords) == 0:
         keywords = ['A', 'B']
    test_words.append('; '.join( keywords))
    

/Users/gwj1139/anaconda3/envs/nlp_whale/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'al', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn(
/Users/gwj1139/anaconda3/envs/nlp_whale/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'al', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn(
/Users/gwj1139/anaconda3/envs/nlp_whale/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Token

KeyboardInterrupt: 

In [1]:
test['Keywords'] = test_words
test[['uuid', 'Keywords']].to_csv('submit_task2.csv', index=None)

NameError: name 'test_words' is not defined

In [22]:

# # 数据清洗
import pandas as pd
import re


# 1. 读入并去除非字母的数字符号
df1 = pd.read_csv('./docx/submit_word40.csv')
df2 = pd.read_csv('./docx/submit_keyword3.csv')
# df['Keywords'] = df['Keywords'].apply(lambda x: re.sub(r'\W+', '', x))
# df['Keywords'] = df['Keywords'].str.replace(r'\W+', '')

# 2. 读入后去掉只有数字或符号构成的无意义的词
import re
def remove_meaningless_words(keywords_str):
    # 定义正则表达式，匹配只有数字或符号构成的无意义的词
    pattern = r'\b(?!\d+\b|\W+\b)\w+\b'
    # 使用findall函数找到所有符合要求的词，并使用join函数将它们拼接成一个字符串
    cleaned_str = ';'.join(re.findall(pattern, keywords_str))
    # 返回清理后的字符串
    return cleaned_str
def remove_meaningless_words2(keywords_str):
    # 定义正则表达式，匹配只有数字或符号构成的无意义的词
    pattern = r'\b(?!\d+\b|\W+\b)\w+\b'
    # 使用findall函数找到所有符合要求的词，并使用join函数将它们拼接成一个字符串
    cleaned_words = re.findall(pattern, keywords_str)
    # 将相邻的单词合并成词组
    cleaned_words_with_group = []
    current_group = []
    for word in cleaned_words:
        if ' ' in word:
            # 如果当前单词包含空格，说明它是词组的一部分，将其保存到临时列表中
            current_group.append(word)
        elif not current_group:
            # 如果当前单词不是词组的一部分，并且当前没有正在处理的词组，将其作为一个单独的单词保存
            cleaned_words_with_group.append(word)
        else:
            # 如果当前单词不是词组的一部分，并且当前正在处理词组，将当前单词和之前的所有单词组成一个词组
            current_group.append(word)
            cleaned_words_with_group.append(' '.join(current_group))
            current_group = []
    if current_group:
        # 在处理完所有单词之后，如果仍有未处理的词组，将其作为一个词组保存
        cleaned_words_with_group.append(' '.join(current_group))
    # 使用join函数将词组拼接成一个字符串，词组之间使用分号分隔
    cleaned_str = ';'.join(cleaned_words_with_group)
    # 返回清理后的字符串
    return cleaned_str
df1['Keywords'] = df1['Keywords'].apply(remove_meaningless_words)
# df2['Keywords'] = df2['Keywords'].apply(remove_meaningless_words2)

# print(df2['Keywords'][0])
# print(df2['Keywords'])

# 3. 取对应的词合并
# df1和df2都是pandas数据结构，他们都有Keywords列，列中的数据格式如下：
# impact smoking intensity; concentrations smokers smoking; prospective population based; 是用封号分隔的若干个关键词。
# 现在要将df2中Keywords列的前10个关键词取出，放在df1的Keywords列的后面，合成新的df3。
# 请用python实现。
def extract_keywords(keywords_str):
    keywords = keywords_str.split(';')
    return ';'.join(keywords[:20])

df1_keywords_20 = df1['Keywords'].apply(extract_keywords)
df2_keywords_20 = df2['Keywords'].apply(extract_keywords)

df3 = pd.concat([df1, df2], axis=0)
df3['Keywords10'] = df1_keywords_20
df3['Keywords20'] = df2_keywords_20

# df3都是pandas数据结构，有Keywords和Keywords10列，列中的数据格式如下：
# impact smoking intensity; concentrations smokers smoking; prospective population based; 是用封号分隔的若干个关键词。
# 现在要将两列合并为keywords_new列，删去原来的两列。
# 请用python实现。
def combine_keywords(row):
    return row['Keywords10'] + ';' + row['Keywords20']

# 将Keywords和Keywords10列合并为keywords_new列
df3['keywords_new'] = df3.apply(combine_keywords, axis=1)
df3.drop(['Keywords10', 'Keywords20'], axis=1, inplace=True)

df3.to_csv("docx/submit_keyword3_3.csv")
df3.head(5)



,uuid,Keywords,label,keywords_new
0,0,dependent;ci;cessation;associations;impact;hun...,1,dependent;ci;cessation;associations;impact;hun...
1,1,company;examination;policy;facebook;scope;aspe...,1,company;examination;policy;facebook;scope;aspe...
2,2,study;implicit;path;positively;controlling;mat...,0,study;implicit;path;positively;controlling;mat...
3,3,develop;success;potential;discover;stakeholder...,1,develop;success;potential;discover;stakeholder...
4,4,contribution;activity;activities;components;li...,1,contribution;activity;activities;components;li...


In [29]:
# !pip install chardet
import csv
import chardet

with open('docx/submit_keyword3_2.csv', 'rb') as f:
    result = chardet.detect(f.read())

encoding = result['encoding']

with open('docx/submit_keyword3_2.csv', encoding=encoding) as f:
    reader = csv.reader(f)
    for row in reader:
        try:
            row[0]
        except UnicodeDecodeError:
            print("Non-UTF character found in row:", row)

